In [2]:
import json
import pandas as pd
from pprint import pprint
data = open("input_csv/ed_trade_data.json")
data = json.load(data)

In [15]:
l = list(data.keys())
l2 = []
for i in l:
    x_i = data[i]
    df_i =  pd.DataFrame(x_i)
    df_i['datetime'] = pd.to_datetime(df_i['date'])
    df_i['price'] = df_i['price'].astype(float)
    df_i['amount'] = df_i['amount'].astype(float)
    df_i['amountBase'] = df_i['amountBase'].astype(float)
    df = df_i[['datetime', 'amount', 'amountBase', 'price']].set_index('datetime')
    pair = i
    row, col = df_i.shape
    maxdate = df.index.max()
    mindate = df.index.min()
    total_days = (maxdate - mindate).days
    grouped = df.groupby(lambda x : x.month)
    grouped.index = 'month'
    a = grouped.count()
    grouped = df.groupby(lambda x: x.month)
    grouped.index = 'month'
    no_of_trades = (grouped['amount'].count())
    a = grouped['amount'].sum()
    b = grouped['amountBase'].sum()
    c = (b / a)
    if (total_days != 0):
        daily_trades_average = sum(no_of_trades) / (total_days)
    else:
        daily_trades_average = 0
    result = [pair, row, mindate, maxdate, total_days, sum(no_of_trades), sum(a), sum(b), (sum(c)/len(c)), daily_trades_average]
    l2.append(result)
output = pd.DataFrame(l2)
output.columns = ['pair', 'no_of_rows', 'min_date', 'max_date', 'total_days', 'no_of_trades', 'volume_traded', 'total_turnover', 'avg_price_of_unit_volume', 'avg_daily_trades']
resultant = output.set_index('pair')
resultant

,no_of_rows,min_date,max_date,total_days,no_of_trades,volume_traded,total_turnover,avg_price_of_unit_volume,avg_daily_trades
pair,,,,,,,,,
ETH_NEWB,353,2017-08-17 03:59:39,2017-11-02 13:31:54,77,353,1.592915e+06,58.522860,3.567360e-05,4
ETH_NTC,500,2017-10-07 04:18:37,2017-11-02 13:04:45,26,500,3.294433e+04,244.158512,6.304795e-03,19
ETH_LNK,271,2017-08-17 00:23:24,2017-11-01 02:11:25,76,271,1.364495e+03,155.122273,1.041670e-01,3
ETH_IXT,500,2017-10-16 17:33:49,2017-11-02 11:55:33,16,500,7.868543e+05,493.627737,6.466308e-04,31
ETH_ETHP,177,2017-10-17 12:47:02,2017-11-02 10:47:24,15,177,5.989485e+05,14.015264,2.582241e-05,11
ETH_SUB,500,2017-10-24 20:54:45,2017-11-02 10:11:41,8,500,8.889812e+05,382.461776,4.114484e-04,62
ETH_1ST,297,2017-08-18 03:08:49,2017-11-02 08:30:48,76,297,9.278350e+04,114.536451,1.856293e-03,3
ETH_ETHG,408,2017-10-17 09:26:14,2017-11-02 03:29:11,15,408,7.730317e+05,66.549733,6.893105e-05,27
ETH_ETHD,500,2017-10-29 22:14:08,2017-11-02 12:16:46,3,500,5.051058e+05,318.079105,6.573749e-04,166


In [4]:
resultant.dtypes

no_of_rows                           int64
min_date                    datetime64[ns]
max_date                    datetime64[ns]
total_days                           int64
no_of_trades                         int64
volume_traded                      float64
total_turnover                     float64
avg_price_of_unit_volume           float64
avg_daily_trades                     int64
dtype: object

In [5]:
s = resultant.sort_values(by = ['no_of_rows', 'total_days', 'volume_traded', 'total_turnover'], ascending = [False, False, False, False])
s

,no_of_rows,min_date,max_date,total_days,no_of_trades,volume_traded,total_turnover,avg_price_of_unit_volume,avg_daily_trades
pair,,,,,,,,,
ETH_SAN,500,2017-08-18 06:37:21,2017-11-02 14:44:32,76,500,5.724847e+05,729.862908,9.867849e-04,6
ETH_NXX,500,2017-08-20 03:55:41,2017-11-02 13:39:47,74,500,4.175280e+06,114.344712,2.857511e-05,6
ETH_BAS,500,2017-08-21 09:46:06,2017-11-02 07:21:44,72,500,1.965430e+05,107.435947,3.532896e-04,6
ETH_MyB,500,2017-08-22 13:01:09,2017-11-02 13:21:06,72,500,3.074222e+04,252.641329,7.597829e-03,6
ETH_OAX,500,2017-08-26 15:16:09,2017-11-02 00:43:23,67,500,1.839338e+05,341.466301,1.943950e-03,7
ETH_STORJ,500,2017-08-26 12:35:50,2017-11-02 11:55:54,67,500,1.130058e+05,202.154446,1.967395e-03,7
ETH_BRAT,500,2017-08-28 21:13:34,2017-11-02 09:32:06,65,500,1.200371e+07,21.256353,1.462727e-06,7
ETH_ECN,500,2017-09-04 06:11:13,2017-11-02 10:14:49,59,500,1.478313e+05,987.392473,6.835232e-03,8
ETH_BLX,500,2017-09-04 13:43:41,2017-11-02 10:49:09,58,500,1.247101e+05,637.595014,4.950690e-03,8


In [25]:
s1 = s.query('no_of_trades >= 500')
top_50_pairs = s1.sort_values(by = ['avg_daily_trades'], ascending = False)
top_50_pairs.head(50).to_html('output_files/top_50_pairs.html')

In [ ]:
def get_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3 - q1 #Interquartile range
    fence_low  = q1 - 1.5 * iqr
    fence_high = q3 + 1.5 * iqr
    df_out = df_in.loc[(df_in[col_name] < fence_low) | (df_in[col_name] > fence_high)]
    return df_out[col_name]
get_outlier(sdata, 'price')
